In [1]:
from pylab import plt
import pandas as pd
import numpy as np
import sympy as sy
import matplotlib.pyplot as plt
import datetime as dt
import scipy as sc
from pylab import mpl
import scipy.integrate as sci
import scipy.stats as scs
import pandas_datareader as web
import statsmodels.api as sm
import enum
import scipy.optimize as spo
from numpy.fft import *
from yahoo_fin import options
from dawp import *
from mystic.solvers import fmin, fmin_powell, diffev, diffev2, sparsity,lattice
import math
import sys
import yfinance as yf

sys.path.append('../')
sys.path.append('C:/Users/kiedy/dx')

import autograd.numpy as agnp
from autograd import grad

np.set_printoptions(suppress=True, formatter={'all': lambda x: '%5.4f' % x})

In [2]:
options1=pd.DataFrame([43.10, 35.60, 22.90, 20.20, 13.30],
                     columns=['price'])
options1['strike']=(1090,1100,1120,1125,1140)
options1['maturity']=dt.datetime(2002,5,17)
options1['T_2']=0.088


options2=pd.DataFrame([84.50, 64.30, 39.50, 33.50, 30.70, 28.00,  25.60, 23.20,19.10,15.30,12.10,10.90],
                     columns=['price'])
options2['strike']=(1050,1075,1110,1120,1125,1130,1135,1140,1150,1160,1170,1175)
options2['maturity']=dt.datetime(2002,6,21)
options2['T_2']=0.184


options3=pd.DataFrame([161.60, 144.80, 120.10, 100.70, 82.50,65.50,51.00,45.50,38.10,27.70,19.60,13.20],
                     columns=['price'])
options3['strike']=(975,995,1025,1050,1075,1100,1125,1135,1150,1175,1200,1225)
options3['maturity']=dt.datetime(2002,9,20)
options3['T_2']=0.436



options4=pd.DataFrame([173.30,157.00,133.10,114.80,97.60,81.20,66.90,58.90,53.90,42.50,33.00,24.90,18.30,13.20],
                     columns=['price'])
options4['strike']=(975,995,1025,1050,1075,1100,1125,1140,1150,1175,1200,1225,1250,1275)
options4['maturity']=dt.datetime(2002,12,20)
options4['T_2']=0.692


options5=pd.DataFrame([146.50,96.20,81.70,68.30,56.60,46.10,36.90,29.30,22.50,17.20,12.80],
                     columns=['price'])
options5['strike']=(1025,1100,1125,1150,1175,1200,1225,1250,1275,1300,1325)
options5['maturity']=dt.datetime(2003,3,21)
options5['T_2']=0.936


options6=pd.DataFrame([182.10,143.00,111.30,97.00,83.30,60.90,49.80,41.20,27.10,17.10,10.10],
                     columns=['price'])
options6['strike']=(995,1050,1100,1125,1150,1200,1225,1250,1300,1350,1400)
options6['maturity']=dt.datetime(2003,6,20)
options6['T_2']=1.192

options7=pd.DataFrame([171.40,140.40,112.80,99.80,66.90,49.50,35.70,25.20,17.00,12.20],
                     columns=['price'])
options7['strike']=(1050,1100,1150,1175,1250,1300,1350,1400,1450,1500)
options7['maturity']=dt.datetime(2003,12,19)
options7['T_2']=1.708

frames=[options1, options2, options3, options4, options5, options6, options7]

options=pd.concat(frames, ignore_index=True)
options['pricing_date'] = dt.datetime(2002, 4, 18)
options['T']=(options['maturity']-options['pricing_date']).dt.days/365.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


r=0.019
q=0.012
S0=1124.47
i = np.complex(0.0, 1.0)

# Metoda COS

In [3]:
def cf(u, T, r, alpha, beta, delta):
    ''' Funkcja charakterystyczna w modelu Kou '''
    b = r - q - 2 * delta * (np.log(np.cos(beta/2))-np.log(np.cos((alpha+beta)/2)))
    value = np.exp(1j * u * b * T)*(np.cos(beta/2)/np.cosh((alpha*u-1j*beta)/2))**(2*delta*T)
    return value

In [4]:
def G(u, *args):
    ''' Funkcja generująca kumulanty'''
    return np.log(cf(-1j*u,T,r, alpha, beta, delta))

T=1
alpha=0.1
beta=0.1
delta=0.2

In [5]:
N=2**12
L=10
h=1e-4
    
def CallPutOptionPriceCOSMthd(cf,S0,r,T,K,alpha, beta, delta,N,L,CALL):


    # cf   - Characteristic function is a function, in the book denoted by \varphi
    # CP   - C for call and P for put
    # S0   - Initial stock price
    # r    - Interest rate (constant)
    # T    - Time to maturity
    # K    - List of strikes
    # N    - Number of expansion terms
    # L    - Size of truncation domain (typ.:L=8 or L=10)
        
    # Reshape K to become a column vector

    if K is not np.array:
        K = np.array(K).reshape([len(K),1])
        
    x0 = np.log(S0 / K)
    
    # kumulanty
    c1=T*(r-q-2*delta*np.log(np.cos(beta/2))-np.log(np.cos((alpha+beta)/2)))+T*alpha*delta*np.tan(beta/2)
    c2=0.5*T*alpha**2*delta/((np.cos(beta/2))**2)
    c4=0.25*T*alpha**4*delta*(2-np.cos(beta))/((np.cos(beta/2))**4)
    
    # kumulanty z automatycznego różniczkowania
#     c1=grad_g(0.)
#     c2=grad_g2(0.)
#     c4=grad_g4(0.)

    # kumulanty z metody różnic skończonych
#     c1=(G(h)-G(-h))/(2*h)
#     c2=(G(h)-2*G(0)+G(-h))/(h**2)
#     c4=(G(3*h)-2*G(2*h)+4*G(0)-G(h)-G(-h)-2*G(-2*h)+G(-3*h))/(4*h**4)

    a=c1+x0-L*np.sqrt(c2+np.sqrt(c4))
    b=c1+x0+L*np.sqrt(c2+np.sqrt(c4))
    # Summation from k = 0 to k=N-1

    k = np.linspace(0,N-1,N).reshape([N,1])  
    u = k * np.pi / (b - a);  

    # Determine coefficients for put prices  

    H_k = CallPutCoefficients(a,b,k)   
    mat = np.exp(1j * np.outer((x0 - a) , u))
    temp = cf(u, T,r, alpha, beta, delta) * H_k 
    temp[0] = 0.5 * temp[0]    
    value = np.exp(-r * T) * K * np.real(mat.dot(temp))
    if CALL == True:
        return  value + np.exp(-q*T)*S0 - K*np.exp(-r*T)
    else:
        return value

# Determine coefficients for put prices 

def CallPutCoefficients(a,b,k):
    
    c = a
    d = 0.0
    coef = Chi_Psi(a,b,c,d,k)
    Chi_k = coef["chi"]
    Psi_k = coef["psi"]
    if a<b and b<0.0:
        d = b
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        H_k      = 2.0 / (b - a) * (- Chi_k + Psi_k) 
    elif a>0 and a<b:
        H_k = np.zeros([len(k),1])
    else:
        H_k = 2.0 / (b - a) * (- Chi_k + Psi_k)
    
    return H_k    

def Chi_Psi(a,b,c,d,k):
    psi = np.sin(k * np.pi * (d - a) / (b - a)) - np.sin(k * np.pi * (c - a)/(b - a))
    psi[1:] = psi[1:] * (b - a) / (k[1:] * np.pi)
    psi[0] = d - c
    
    chi = 1.0 / (1.0 + np.power((k * np.pi / (b - a)) , 2.0)) 
    expr1 = np.cos(k * np.pi * (d - a)/(b - a)) * np.exp(d)  - np.cos(k * np.pi 
                  * (c - a) / (b - a)) * np.exp(c)
    expr2 = k * np.pi / (b - a) * np.sin(k * np.pi * 
                        (d - a) / (b - a))   - k * np.pi / (b - a) * np.sin(k 
                        * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi = chi * (expr1 + expr2)
    
    value = {"chi":chi,"psi":psi }
    return value

In [6]:
m=0
def Meixner_error_function(p0):
    
    global m
    alpha, beta, delta=p0
    if beta<=-np.pi or beta>np.pi or delta<=0 or alpha<=0 or np.cos((alpha+beta)/2)<=0:
        return 100
    
    se = []
    for row, option in options.iterrows():
        model_value = CallPutOptionPriceCOSMthd(cf,S0,r,option['T_2'],(option['strike'],),alpha, beta, delta,N,L, True)[0][0]
        if np.isnan(model_value):
            model_value=100000
        se.append((model_value - option['price']) ** 2)
    RMSE = (sum(se) / len(se))**0.5
    if m % 1 == 0:
        print('%4d |' % m, np.array(p0), '| %7.3f' % RMSE)
        
    m += 1
    return RMSE

Meixner_error_function((0.3917 ,-1.4912 ,0.3538))

   0 | [0.3917 -1.4912 0.3538] |   3.237


3.2366173110480236

# Kalibracja dla wielu K

In [10]:
N=2**12
L=10
h=1e-3
    
def CallPutOptionPriceCOSMthd(cf,S0,r,T,K,alpha, beta, delta,N,L,CALL):


    # cf   - Characteristic function is a function, in the book denoted by \varphi
    # CP   - C for call and P for put
    # S0   - Initial stock price
    # r    - Interest rate (constant)
    # T    - Time to maturity
    # K    - List of strikes
    # N    - Number of expansion terms
    # L    - Size of truncation domain (typ.:L=8 or L=10)
        
    # Reshape K to become a column vector

    if K is not np.array:
        K = np.array(K).reshape([len(K),1])
        
    x0 = np.log(S0 / K)
    
    # kumulanty
    c1=T*(r-q-2*delta*np.log(np.cos(beta/2))-np.log(np.cos((alpha+beta)/2)))+T*alpha*delta*np.tan(beta/2)
    c2=0.5*T*alpha**2*delta/((np.cos(beta/2))**2)
    c4=0.25*T*alpha**4*delta*(2-np.cos(beta))/((np.cos(beta/2))**4)
    
    # kumulanty z automatycznego różniczkowania
#     c1=grad_g(0.)
#     c2=grad_g2(0.)
#     c4=grad_g4(0.)

    # kumulanty z metody różnic skończonych
#     c1=(G(h)-G(-h))/(2*h)
#     c2=(G(h)-2*G(0)+G(-h))/(h**2)
#     c4=(G(3*h)-2*G(2*h)+4*G(0)-G(h)-G(-h)-2*G(-2*h)+G(-3*h))/(4*h**4)

    a=c1-L*np.sqrt(c2+np.sqrt(c4))
    b=c1+L*np.sqrt(c2+np.sqrt(c4))
    # Summation from k = 0 to k=N-1

    k = np.linspace(0,N-1,N).reshape([N,1])  
    u = k * np.pi / (b - a);  

    # Determine coefficients for put prices  

    H_k = CallPutCoefficients(a,b,k)   
    mat = np.exp(1j * np.outer((x0 - a) , u))
    temp = cf(u, T,r,alpha, beta, delta) * H_k 
    temp[0] = 0.5 * temp[0]    
    value = np.exp(-r * T) * K * np.real(mat.dot(temp))
    if CALL == True:
        return  value + np.exp(-q*T)*S0 - K*np.exp(-r*T)
    else:
        return value

# Determine coefficients for put prices 

def CallPutCoefficients(a,b,k):
    
    c = a
    d = 0.0
    coef = Chi_Psi(a,b,c,d,k)
    Chi_k = coef["chi"]
    Psi_k = coef["psi"]
    if a<b and b<0.0:
        d = b
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        H_k      = 2.0 / (b - a) * (- Chi_k + Psi_k) 
    elif a>0 and a<b:
        H_k = np.zeros([len(k),1])
    else:
        H_k = 2.0 / (b - a) * (- Chi_k + Psi_k)
    
    return H_k    

def Chi_Psi(a,b,c,d,k):
    psi = np.sin(k * np.pi * (d - a) / (b - a)) - np.sin(k * np.pi * (c - a)/(b - a))
    psi[1:] = psi[1:] * (b - a) / (k[1:] * np.pi)
    psi[0] = d - c
    
    chi = 1.0 / (1.0 + np.power((k * np.pi / (b - a)) , 2.0)) 
    expr1 = np.cos(k * np.pi * (d - a)/(b - a)) * np.exp(d)  - np.cos(k * np.pi 
                  * (c - a) / (b - a)) * np.exp(c)
    expr2 = k * np.pi / (b - a) * np.sin(k * np.pi * 
                        (d - a) / (b - a))   - k * np.pi / (b - a) * np.sin(k 
                        * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi = chi * (expr1 + expr2)
    
    value = {"chi":chi,"psi":psi }
    return value

In [11]:
m=0
def Meixner_error_function(p0):
    
    global m
    alpha, beta, delta = p0
#     if sigma<=0:
#         return 100
    se1 = []
    K = np.array(options1['strike'])
    T=options['T_2'][0]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,alpha, beta, delta, N,L, True)
    i=0
    for row, option in options1.iterrows():
        se1.append((model_value[i] - option['price']) ** 2)
        i+=1
        
        
    se2 = []
    K = np.array(options2['strike'])
    T=options['T_2'][10]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,alpha, beta, delta,N,L, True)
    i=0
    for row, option in options2.iterrows():
        se2.append((model_value[i] - option['price']) ** 2)
        i+=1
        
        
    se3 = []    
    K = np.array(options3['strike'])
    T=options['T_2'][18]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,alpha, beta, delta,N,L, True)
    i=0
    for row, option in options3.iterrows():
        se3.append((model_value[i] - option['price']) ** 2)
        i+=1        
        
        
    se4 = []    
    K = np.array(options4['strike'])
    T=options['T_2'][37]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,alpha, beta, delta,N,L, True)
    i=0
    for row, option in options4.iterrows():
        se4.append((model_value[i] - option['price']) ** 2)
        i+=1        

    se5 = []    
    K = np.array(options5['strike'])
    T=options['T_2'][48]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,alpha, beta, delta,N,L, True)
    i=0
    for row, option in options5.iterrows():
        se5.append((model_value[i] - option['price']) ** 2)
        i+=1 
        
    se6 = []    
    K = np.array(options6['strike'])
    T=options['T_2'][58]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,alpha, beta, delta,N,L, True)
    i=0
    for row, option in options6.iterrows():
        se6.append((model_value[i] - option['price']) ** 2)
        i+=1
        
        
    se7 = []    
    K = np.array(options7['strike'])
    T=options['T_2'][69]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,T,K,alpha, beta, delta,N,L, True)
    i=0
    for row, option in options7.iterrows():
        se7.append((model_value[i] - option['price']) ** 2)
        i+=1
        
    RMSE = ((sum(se1)+sum(se2)+sum(se3)+sum(se4)+sum(se5)+sum(se6)+sum(se7)) / (len(se1)+len(se2)+len(se3)+len(se4)+len(se5)+len(se6)+len(se7)))**0.5
    if np.isnan(RMSE):
        RMSE=100
        
    if m % 1 == 0:
        print('%4d |' % m, np.array(p0), '| %7.3f' % RMSE)
        
    m += 1
    return RMSE

# def BS_calibration_full():
#     i=0
#     global opt
#     p0 = spo.brute(BS_error_function, ((0.07,0.4,0.1),), finish=None)
#     opt = spo.fmin(BS_error_function, p0, xtol=0.000001, ftol=0.000001, maxiter=None, maxfun=None)
#     return opt

Meixner_error_function((0.3917 ,-1.4912 ,0.3538))


   0 | [0.3917 -1.4912 0.3538] |   3.237


array([3.2366])

In [12]:
def Meixner_calibration_full():
    i=0
    global opt1, opt
    opt1 = spo.brute(Meixner_error_function, ((0.001,1.51, 0.1),(-np.pi,np.pi, 0.3), (0.001,2.,0.16)), finish=None)
    opt = spo.fmin(Meixner_error_function, opt1, xtol=0.00001, ftol=0.00001, maxiter=15000, maxfun=15000)
    return opt

Meixner_calibration_full()

   1 | [0.0010 -3.1416 0.0010] | 488.312
   2 | [0.0010 -3.1416 0.1610] | 886.983
   3 | [0.0010 -3.1416 0.3210] | 967.269
   4 | [0.0010 -3.1416 0.4810] | 997.856
   5 | [0.0010 -3.1416 0.6410] | 1014.082
   6 | [0.0010 -3.1416 0.8010] | 1024.274
   7 | [0.0010 -3.1416 0.9610] | 1031.267
   8 | [0.0010 -3.1416 1.1210] | 1036.299
   9 | [0.0010 -3.1416 1.2810] | 1040.026
  10 | [0.0010 -3.1416 1.4410] | 1042.842
  11 | [0.0010 -3.1416 1.6010] | 1045.001
  12 | [0.0010 -3.1416 1.7610] | 1046.674
  13 | [0.0010 -3.1416 1.9210] | 1047.983
  14 | [0.0010 -2.8416 0.0010] | 119.882
  15 | [0.0010 -2.8416 0.1610] | 119.882
  16 | [0.0010 -2.8416 0.3210] | 119.882
  17 | [0.0010 -2.8416 0.4810] | 119.882
  18 | [0.0010 -2.8416 0.6410] | 119.882
  19 | [0.0010 -2.8416 0.8010] | 119.882
  20 | [0.0010 -2.8416 0.9610] | 119.882
  21 | [0.0010 -2.8416 1.1210] | 119.882
  22 | [0.0010 -2.8416 1.2810] | 119.882
  23 | [0.0010 -2.8416 1.4410] | 119.882
  24 | [0.0010 -2.8416 1.6010] | 119.882
  25 | 

<ipython-input-3-204edf3d47ae>:4: RuntimeWarning: overflow encountered in cosh
  value = np.exp(1j * u * b * T)*(np.cos(beta/2)/np.cosh((alpha*u-1j*beta)/2))**(2*delta*T)
<ipython-input-3-204edf3d47ae>:4: RuntimeWarning: invalid value encountered in true_divide
  value = np.exp(1j * u * b * T)*(np.cos(beta/2)/np.cosh((alpha*u-1j*beta)/2))**(2*delta*T)
<ipython-input-3-204edf3d47ae>:4: RuntimeWarning: overflow encountered in true_divide
  value = np.exp(1j * u * b * T)*(np.cos(beta/2)/np.cosh((alpha*u-1j*beta)/2))**(2*delta*T)


 [0.0010 -2.5416 0.3210] | 119.882
  30 | [0.0010 -2.5416 0.4810] | 119.882
  31 | [0.0010 -2.5416 0.6410] | 119.882
  32 | [0.0010 -2.5416 0.8010] | 119.882
  33 | [0.0010 -2.5416 0.9610] | 119.882
  34 | [0.0010 -2.5416 1.1210] | 119.882
  35 | [0.0010 -2.5416 1.2810] | 119.882
  36 | [0.0010 -2.5416 1.4410] | 119.882
  37 | [0.0010 -2.5416 1.6010] | 119.882
  38 | [0.0010 -2.5416 1.7610] | 119.882
  39 | [0.0010 -2.5416 1.9210] | 119.882
  40 | [0.0010 -2.2416 0.0010] | 100.000
  41 | [0.0010 -2.2416 0.1610] | 119.882
  42 | [0.0010 -2.2416 0.3210] | 119.882
  43 | [0.0010 -2.2416 0.4810] | 119.882
  44 | [0.0010 -2.2416 0.6410] | 119.882
  45 | [0.0010 -2.2416 0.8010] | 119.882
  46 | [0.0010 -2.2416 0.9610] | 119.882
  47 | [0.0010 -2.2416 1.1210] | 119.882
  48 | [0.0010 -2.2416 1.2810] | 119.882
  49 | [0.0010 -2.2416 1.4410] | 119.882
  50 | [0.0010 -2.2416 1.6010] | 119.882
  51 | [0.0010 -2.2416 1.7610] | 119.882
  52 | [0.0010 -2.2416 1.9210] | 119.882
  53 | [0.0010 -1.9416

 233 | [0.0010 1.9584 1.7610] | 119.882
 234 | [0.0010 1.9584 1.9210] | 119.882
 235 | [0.0010 2.2584 0.0010] | 100.000
 236 | [0.0010 2.2584 0.1610] | 119.882
 237 | [0.0010 2.2584 0.3210] | 119.882
 238 | [0.0010 2.2584 0.4810] | 119.882
 239 | [0.0010 2.2584 0.6410] | 119.882
 240 | [0.0010 2.2584 0.8010] | 119.882
 241 | [0.0010 2.2584 0.9610] | 119.882
 242 | [0.0010 2.2584 1.1210] | 119.882
 243 | [0.0010 2.2584 1.2810] | 119.882
 244 | [0.0010 2.2584 1.4410] | 119.882
 245 | [0.0010 2.2584 1.6010] | 119.882
 246 | [0.0010 2.2584 1.7610] | 119.882
 247 | [0.0010 2.2584 1.9210] | 119.882
 248 | [0.0010 2.5584 0.0010] | 100.000
 249 | [0.0010 2.5584 0.1610] | 119.882
 250 | [0.0010 2.5584 0.3210] | 119.882
 251 | [0.0010 2.5584 0.4810] | 119.882
 252 | [0.0010 2.5584 0.6410] | 119.882
 253 | [0.0010 2.5584 0.8010] | 119.882
 254 | [0.0010 2.5584 0.9610] | 119.882
 255 | [0.0010 2.5584 1.1210] | 119.882
 256 | [0.0010 2.5584 1.2810] | 119.882
 257 | [0.0010 2.5584 1.4410] | 119.882


 435 | [0.1010 0.4584 0.8010] | 100.000
 436 | [0.1010 0.4584 0.9610] |  34.176
 437 | [0.1010 0.4584 1.1210] |  32.763
 438 | [0.1010 0.4584 1.2810] |  31.403
 439 | [0.1010 0.4584 1.4410] |  30.090
 440 | [0.1010 0.4584 1.6010] |  28.819
 441 | [0.1010 0.4584 1.7610] |  27.587
 442 | [0.1010 0.4584 1.9210] |  26.389
 443 | [0.1010 0.7584 0.0010] | 100.000
 444 | [0.1010 0.7584 0.1610] | 100.000
 445 | [0.1010 0.7584 0.3210] | 100.000
 446 | [0.1010 0.7584 0.4810] | 100.000
 447 | [0.1010 0.7584 0.6410] | 100.000
 448 | [0.1010 0.7584 0.8010] |  34.862
 449 | [0.1010 0.7584 0.9610] |  33.269
 450 | [0.1010 0.7584 1.1210] |  31.746
 451 | [0.1010 0.7584 1.2810] |  30.283
 452 | [0.1010 0.7584 1.4410] |  28.873
 453 | [0.1010 0.7584 1.6010] |  27.511
 454 | [0.1010 0.7584 1.7610] |  26.192
 455 | [0.1010 0.7584 1.9210] |  24.914
 456 | [0.1010 1.0584 0.0010] | 100.000
 457 | [0.1010 1.0584 0.1610] | 100.000
 458 | [0.1010 1.0584 0.3210] | 100.000
 459 | [0.1010 1.0584 0.4810] | 100.000


...

3946 | [1.4010 -0.4416 0.9610] | 282.031
3947 | [1.4010 -0.4416 1.1210] | 310.270
3948 | [1.4010 -0.4416 1.2810] | 335.973
3949 | [1.4010 -0.4416 1.4410] | 359.581
3950 | [1.4010 -0.4416 1.6010] | 381.421
3951 | [1.4010 -0.4416 1.7610] | 401.743
3952 | [1.4010 -0.4416 1.9210] | 420.741
3953 | [1.4010 -0.1416 0.0010] | 100.000
3954 | [1.4010 -0.1416 0.1610] | 100.000
3955 | [1.4010 -0.1416 0.3210] | 100.000
3956 | [1.4010 -0.1416 0.4810] | 100.000
3957 | [1.4010 -0.1416 0.6410] | 100.000
3958 | [1.4010 -0.1416 0.8010] | 100.000
3959 | [1.4010 -0.1416 0.9610] | 100.000
3960 | [1.4010 -0.1416 1.1210] | 322.581
3961 | [1.4010 -0.1416 1.2810] | 348.872
3962 | [1.4010 -0.1416 1.4410] | 372.985
3963 | [1.4010 -0.1416 1.6010] | 395.262
3964 | [1.4010 -0.1416 1.7610] | 415.961
3965 | [1.4010 -0.1416 1.9210] | 435.287
3966 | [1.4010 0.1584 0.0010] | 100.000
3967 | [1.4010 0.1584 0.1610] | 100.000
3968 | [1.4010 0.1584 0.3210] | 100.000
3969 | [1.4010 0.1584 0.4810] | 100.000
3970 | [1.4010 0.158

4150 | [1.5010 -1.9416 0.3210] | 177.406
4151 | [1.5010 -1.9416 0.4810] | 242.322
4152 | [1.5010 -1.9416 0.6410] | 294.999
4153 | [1.5010 -1.9416 0.8010] | 339.580
4154 | [1.5010 -1.9416 0.9610] | 378.305
4155 | [1.5010 -1.9416 1.1210] | 412.547
4156 | [1.5010 -1.9416 1.2810] | 443.215
4157 | [1.5010 -1.9416 1.4410] | 470.952
4158 | [1.5010 -1.9416 1.6010] | 496.231
4159 | [1.5010 -1.9416 1.7610] | 519.414
4160 | [1.5010 -1.9416 1.9210] | 540.784
4161 | [1.5010 -1.6416 0.0010] | 100.000
4162 | [1.5010 -1.6416 0.1610] | 100.000
4163 | [1.5010 -1.6416 0.3210] | 156.234
4164 | [1.5010 -1.6416 0.4810] | 215.435
4165 | [1.5010 -1.6416 0.6410] | 263.899
4166 | [1.5010 -1.6416 0.8010] | 305.232
4167 | [1.5010 -1.6416 0.9610] | 341.393
4168 | [1.5010 -1.6416 1.1210] | 373.582
4169 | [1.5010 -1.6416 1.2810] | 402.597
4170 | [1.5010 -1.6416 1.4410] | 429.001
4171 | [1.5010 -1.6416 1.6010] | 453.209
4172 | [1.5010 -1.6416 1.7610] | 475.540
4173 | [1.5010 -1.6416 1.9210] | 496.241
4174 | [1.5010 -

4354 | [1.5010 2.5584 1.7610] | 100.000
4355 | [1.5010 2.5584 1.9210] | 100.000
4356 | [1.5010 2.8584 0.0010] | 100.000
4357 | [1.5010 2.8584 0.1610] | 100.000
4358 | [1.5010 2.8584 0.3210] | 100.000
4359 | [1.5010 2.8584 0.4810] | 100.000
4360 | [1.5010 2.8584 0.6410] | 100.000
4361 | [1.5010 2.8584 0.8010] | 100.000
4362 | [1.5010 2.8584 0.9610] | 100.000
4363 | [1.5010 2.8584 1.1210] | 100.000
4364 | [1.5010 2.8584 1.2810] | 100.000
4365 | [1.5010 2.8584 1.4410] | 100.000
4366 | [1.5010 2.8584 1.6010] | 100.000
4367 | [1.5010 2.8584 1.7610] | 100.000
4368 | [1.5010 2.8584 1.9210] | 100.000
4369 | [0.3010 -1.6416 0.4810] |   3.380
4370 | [0.3161 -1.6416 0.4810] |   3.993
4371 | [0.3010 -1.7237 0.4810] |   3.754
4372 | [0.3010 -1.6416 0.5050] |   3.496
4373 | [0.2860 -1.6963 0.4970] |   3.495
4374 | [0.2910 -1.5960 0.5077] |   3.799
4375 | [0.2985 -1.6918 0.4877] |   3.423
4376 | [0.2893 -1.7115 0.4721] |   3.781
4377 | [0.2981 -1.6591 0.4968] |   3.349
4378 | [0.3124 -1.6320 0.4800] 

4558 | [0.3903 -1.4912 0.3558] |   3.237
4559 | [0.3903 -1.4912 0.3559] |   3.237
4560 | [0.3903 -1.4914 0.3558] |   3.237
4561 | [0.3904 -1.4914 0.3556] |   3.237
4562 | [0.3903 -1.4914 0.3558] |   3.237
4563 | [0.3903 -1.4912 0.3559] |   3.237
4564 | [0.3903 -1.4914 0.3557] |   3.237
4565 | [0.3903 -1.4912 0.3558] |   3.237
4566 | [0.3903 -1.4914 0.3558] |   3.237
4567 | [0.3903 -1.4916 0.3558] |   3.237
4568 | [0.3903 -1.4913 0.3558] |   3.237
4569 | [0.3904 -1.4914 0.3557] |   3.237
4570 | [0.3903 -1.4914 0.3558] |   3.237
4571 | [0.3903 -1.4913 0.3558] |   3.237
4572 | [0.3903 -1.4913 0.3558] |   3.237
4573 | [0.3903 -1.4913 0.3558] |   3.237
4574 | [0.3903 -1.4914 0.3558] |   3.237
4575 | [0.3903 -1.4913 0.3558] |   3.237
4576 | [0.3903 -1.4914 0.3558] |   3.237
4577 | [0.3903 -1.4914 0.3558] |   3.237
4578 | [0.3903 -1.4914 0.3558] |   3.237
4579 | [0.3903 -1.4913 0.3558] |   3.237
4580 | [0.3903 -1.4913 0.3558] |   3.237
4581 | [0.3903 -1.4913 0.3558] |   3.237
4582 | [0.3903 -

array([0.3903, -1.4913, 0.3558])